# Observer
> Define a one-to-many dependency between objects tso that when one object changes state, all its dependents are notified and updated automatically.

## Problem
Lets say I am writing an OS module that detects whenever the battery level is below a certain threshold. When that happens it tells the `AppManager` to put unused apps to sleep and tells the screen to dim the brightness.

In [1]:
class BatteryMonitor:
    def __init__(self, screen, app_manager):
        self.screen = screen
        self.app_manager = app_manager
        
    def set_strength(self, val):
        if val < 10:
            print(f"Battery level is low")
            self.screen.dim(val)
            self.app_manager.bg(val)

class Screen:
    def dim(self, battery_level):
        print(f"Decreasing brightness appropriate to battery {battery_level}")
        
class AppManager:
    def bg(self, battery_level):
        print(f"Putting unused apps in background for battery {battery_level}")

In [2]:
sc = Screen()
am = AppManager()
bm = BatteryMonitor(sc, am)
bm.set_strength(3)

Battery level is low
Decreasing brightness appropriate to battery 3
Putting unused apps in background for battery 3


But now `BatteryMonitor` is tightly coupled with `Screen` and `AppManager`, i.e., it has to know which methods to call on them. Moreover, if the `Bluetooth` module and the `Wifi` module also want to turn their radios off if the battery level is too low, then I have to change the `BatteryMonitor` class. How can I avoid this tight coupling?

## Solution

In [11]:
from abc import ABC, abstractmethod

class BatteryObserver(ABC):
    @abstractmethod
    def update(self, battery_level):
        pass
    
class BatteryMonitor:
    def __init__(self):
        self.observers = []
        
    def attach(self, obs):
        self.observers.append(obs)
        
    def set_strength(self, val):
        for obs in self.observers:
            obs.update(val)
    
class AppManager(BatteryObserver):
    def __init__(self, bm: BatteryMonitor):
        bm.attach(self)
        
    def bg(self, battery_level):
        print(f"Putting unused apps in background for battery {battery_level}")
    
    def update(self, battery_level):
        if battery_level < 15:
            self.bg(battery_level)
        
class Screen(BatteryObserver):
    def __init__(self, bm: BatteryMonitor):
        bm.attach(self)
        
    def dim(self, battery_level):
        print(f"Decreasing brightness appropriate to battery {battery_level}")
        
    def update(self, battery_level):
        if battery_level < 10:
            self.dim(battery_level)

In [12]:
bm = BatteryMonitor()
sc = Screen(bm)
am = AppManager(bm)
bm.set_strength(3)

Decreasing brightness appropriate to battery 3
Putting unused apps in background for battery 3
